# Automatyczne pozyskiwanie danych

## Tomasz Rodak

Wykład 3

---

## Ciasteczka (*cookies*)

Ciasteczko to niewielki fragment danych przesyłany przez serwer do przeglądarki użytkownika, który jest następnie zapisywany na jego urządzeniu. Przy kolejnych żądaniach przeglądarka wysyła te dane z powrotem do serwera. Umożliwia to:

- **Utrzymanie sesji:** Identyfikacja użytkownika po zalogowaniu oraz utrzymanie stanu sesji.
- **Personalizację:** Zapamiętanie preferencji i ustawień użytkownika na stronie.
- **Śledzenie:** Analiza zachowań użytkowników, m.in. przy użyciu narzędzi analitycznych i reklamowych.

### Mechanizm działania ciasteczek w komunikacji klient-serwer

1.  **Żądanie serwera:**
    * Gdy użytkownik po raz pierwszy odwiedza stronę internetową, jego przeglądarka wysyła żądanie HTTP do serwera, na którym hostowana jest ta strona.
    * Serwer przetwarza to żądanie i generuje odpowiedź HTTP, która zawiera żądaną stronę internetową.
    * W nagłówku odpowiedzi HTTP serwer może dołączyć polecenie `Set-Cookie`, które zawiera informacje o ciasteczku, które ma być zapisane w przeglądarce użytkownika.

2.  **Ustawienie ciasteczka:**
    * Przeglądarka użytkownika odbiera odpowiedź HTTP i analizuje nagłówek.
    * Jeśli w nagłówku znajduje się polecenie `Set-Cookie`, przeglądarka zapisuje ciasteczko na urządzeniu użytkownika.
    
3.  **Kolejne żądania:**
    * Przy każdym kolejnym żądaniu wysyłanym do tego samego serwera, przeglądarka automatycznie dołącza zapisane ciasteczka do nagłówka żądania HTTP.
    * Serwer odbiera żądanie i analizuje nagłówek, aby odczytać zawartość ciasteczek.
    * Na podstawie informacji zawartych w ciasteczkach serwer może dostosować odpowiedź, np. wyświetlić spersonalizowane treści lub utrzymać sesję użytkownika. Przykładowo, jeśli ciasteczko zawiera identyfikator sesji, serwer może zidentyfikować użytkownika i uznać go za zalogowanego.

4.  **Wygaśnięcie ciasteczka:**
    * Każde ciasteczko ma określony czas życia, który jest określony przez datę wygaśnięcia, lub jest to sesyjne ciasteczko, które jest usuwane po zamknięciu przeglądarki.
    * Po upływie tego czasu przeglądarka automatycznie usuwa ciasteczko z urządzenia użytkownika.
    * Ciasteczka mogą również zostać usunięte przez użytkownika ręcznie.

### `Set-Cookie`

Nagłówek **Set-Cookie** umożliwia serwerowi wysłanie do klienta instrukcji, aby zapisał ciasteczko.

### Kluczowe składniki nagłówka Set-Cookie

- **Para klucz=wartość**  
  To podstawowy element ciasteczka, który przechowuje dane (np. `session_id=abc123`).

- **Atrybut Path**  
  Określa ścieżkę w obrębie domeny, dla której ciasteczko będzie dostępne. Jeśli nie zostanie podany, domyślnie przypisywana jest bieżąca ścieżka, z której otrzymano nagłówek.

- **Atrybut Domain**  
  Definiuje, dla jakiej domeny ciasteczko ma być dostępne. Pozwala to na używanie ciasteczek w ramach głównej domeny i jej subdomen.

- **Atrybut Expires / Max-Age**  
  - **Expires**: Ustala datę i godzinę, do kiedy ciasteczko będzie ważne. Po tym terminie ciasteczko wygasa.
  - **Max-Age**: Definiuje czas życia ciasteczka w sekundach, liczony od momentu jego ustawienia.

- **Atrybut Secure**  
  Wskazuje, że ciasteczko ma być przesyłane jedynie przez bezpieczne połączenie (HTTPS), co zwiększa ochronę przed podsłuchiwaniem.

- **Atrybut HttpOnly**  
  Zapobiega dostępowi do ciasteczka przez skrypty po stronie klienta, takie jak JavaScript. To działanie zwiększa bezpieczeństwo, chroniąc ciasteczko przed atakami typu XSS.

- **Atrybut SameSite**  
  Kontroluje, czy ciasteczko ma być wysyłane przy żądaniach cross-site (czyli z innych stron). Może przyjmować wartości:
  - **Strict** – ciasteczko nie jest wysyłane przy żadnych żądaniach z innego źródła.
  - **Lax** – ciasteczko jest wysyłane przy żądaniach nawigacyjnych, ale nie przy żądaniach asynchronicznych.
  - **None** – ciasteczko jest wysyłane przy wszystkich żądaniach, ale przy tej opcji wymagane jest również ustawienie atrybutu Secure.

#### Przykładowa składnia

```
Set-Cookie: session_id=abc123; Path=/; Domain=example.com; Expires=Wed, 21 Oct 2025 07:28:00 GMT; Secure; HttpOnly; SameSite=Strict
```

W tym przykładzie serwer ustawia ciasteczko `session_id` o wartości `abc123`, które:
- Będzie dostępne dla całej domeny (`Path=/`),
- Dotyczy domeny `example.com`,
- Wygasa 21 października 2025 o 07:28:00 GMT,
- Będzie przesyłane tylko przez bezpieczne połączenie (`Secure`),
- Nie będzie dostępne dla skryptów po stronie klienta (`HttpOnly`),
- Nie będzie wysyłane przy żądaniach cross-site (`SameSite=Strict`).

### `Cookie`

Nagłówek **Cookie** jest używany przez klienta (zazwyczaj przeglądarkę) do przesyłania wcześniej zapisanych ciasteczek do serwera w ramach żądania HTTP.

#### Składnia 

Nagłówek `Cookie` zawiera pary klucz=wartość, które są rozdzielone średnikami. Przykładowa składnia wygląda następująco:

```
Cookie: session_id=abc123; theme=dark; lang=pl
```

- **`session_id=abc123`:** Może identyfikować unikalną sesję użytkownika.
- **`theme=dark`:** Informuje o preferowanym motywie (ciemnym) wyświetlania strony.
- **`lang=pl`:** Określa preferowany język interfejsu.

#### Zasada działania

1. **Odbiór ciasteczek:** Po otrzymaniu odpowiedzi z nagłówkiem `Set-Cookie`, przeglądarka zapisuje ciasteczka.
2. **Wysyłka przy kolejnych żądaniach:** Przy kolejnych żądaniach do tej samej domeny, przeglądarka dołącza nagłówek `Cookie` z odpowiednimi wartościami, co umożliwia serwerowi kontynuowanie sesji lub personalizację odpowiedzi.
3. **Brak dodatkowych atrybutów:** W odróżnieniu od nagłówka `Set-Cookie`, nagłówek `Cookie` nie zawiera atrybutów takich jak `Path`, `Domain`, `Secure`, `HttpOnly` czy `SameSite` – wszystkie te ustawienia były wcześniej skonfigurowane przez serwer.

### `http.cookies`

Moduł **`http.cookies`** w Pythonie dostarcza narzędzi do obsługi ciasteczek HTTP, umożliwiając ich tworzenie, modyfikację oraz parsowanie. 

Podstawowe definiowane klasy:
- **`SimpleCookie`**: Umożliwia łatwe zarządzanie zestawem ciasteczek, działając jak słownik.
- **`Morsel`**: Reprezentuje pojedyncze ciasteczko, przechowując jego wartość oraz atrybuty.


#### **`SimpleCookie`**

Klasa, która dziedziczy po słowniku (`dict`) i pozwala na łatwe przechowywanie i operowanie na wielu ciasteczkach. Umożliwia ona dodawanie nowych ciasteczek, modyfikację istniejących oraz generowanie poprawnych nagłówków HTTP, które można wysłać w odpowiedzi serwera.

- **Funkcjonalności:**  
  - **Parsowanie ciasteczek:** Możesz przekazać łańcuch znaków (np. zawartość nagłówka `Cookie`), a `SimpleCookie` rozdzieli poszczególne ciasteczka i przypisze je do kluczy w słowniku.  
  - **Dodawanie ciasteczek:** Umożliwia dodanie nowego ciasteczka, np. `cookie["session"] = "abc123"`.  
  - **Ustawianie atrybutów:** Po utworzeniu ciasteczka można ustawić dodatkowe atrybuty, takie jak `path`, `domain`, `expires`, `secure`, `httponly`, czy `samesite`, np. `cookie["session"]["Max-Age"] = 3600`. 
  - **Generowanie nagłówków:** Po ustawieniu ciasteczek, metoda `output()` generuje poprawnie sformatowany nagłówek `Set-Cookie`, który można bezpośrednio wykorzystać w odpowiedzi HTTP.

- **Przykład użycia:**

  ```python
  from http.cookies import SimpleCookie

  # Utworzenie obiektu SimpleCookie
  cookie = SimpleCookie()
  cookie["session_id"] = "abc123"
  cookie["session_id"]["path"] = "/"
  cookie["session_id"]["httponly"] = True

  # Wygenerowanie nagłówka Set-Cookie
  print(cookie.output())
  ```
  Wyjście:
  ```
  Set-Cookie: session_id=abc123; HttpOnly; Path=/
  ```

### **Morsel**
  
Klasa **`Morsel`** reprezentuje pojedyncze ciasteczko. Jest to struktura danych, która przechowuje zarówno wartość ciasteczka, jak i dodatkowe informacje (atrybuty), takie jak `expires`, `domain`, `path`, `secure` czy `httponly`.

**Funkcjonalności:**  
  - **Przechowywanie danych:** Każdy obiekt Morsel zawiera klucz (nazwę ciasteczka), wartość (surową oraz zakodowaną) oraz słownik atrybutów, który określa dodatkowe właściwości ciasteczka.  
  - **Formatowanie:** Morsel jest odpowiedzialny za poprawne sformatowanie ciasteczka, gdy jest konwertowany do ciągu znaków.  
  - **Walidacja:** Przed ustawieniem wartości lub atrybutów, Morsel może wykonywać pewne operacje walidacyjne, aby upewnić się, że dane są zgodne z wymaganiami protokołu HTTP.

- **Powiązanie z SimpleCookie:**  
  Dla ciasteczka jako obiektu typu `SimpleCookie` istnieje pod spodem obiekt typu Morsel reprezentujący to ciasteczko. 

### Przykład

```python
from http.server import BaseHTTPRequestHandler, HTTPServer
from http.cookies import SimpleCookie
import uuid

# Przechowywanie sesji w pamięci
session_storage = {}

class SimpleHandler(BaseHTTPRequestHandler):
    
    def do_GET(self):
        if self.path == "/":
            # Sprawdzanie czy klient wysyła ciasteczko sesyjne
            cookie_header = self.headers.get("Cookie")
            cookies = SimpleCookie(cookie_header)
            
            # Pobieranie lub tworzenie identyfikatora sesji
            if "session_id" in cookies:
                session_id = cookies["session_id"].value
                # Pobieranie i zwiększanie licznika odwiedzin
                counter = session_storage.get(session_id, 0) + 1
            else:
                # Tworzenie nowej sesji
                session_id = str(uuid.uuid4())
                counter = 1
            
            # Zapisywanie licznika w słowniku sesji
            session_storage[session_id] = counter
            
            # Ustawianie nagłówków odpowiedzi
            self.send_response(200)
            self.send_header("Content-type", "text/html")
            self.send_header("Set-Cookie", f"session_id={session_id}")
            self.end_headers()
            
            # Wczytywanie i modyfikowanie strony HTML
            try:
                with open("page.html", "r") as file:
                    outhtml = file.read()
                outhtml = outhtml.format(counter=counter)
                self.wfile.write(outhtml.encode("utf-8"))
            except FileNotFoundError:
                # Komunikat w razie braku pliku HTML
                html = f"<html><body><h1>Odwiedziny: {counter}</h1></body></html>"
                self.wfile.write(html.encode("utf-8"))

# Uruchomienie serwera
httpd = HTTPServer(('localhost', 21000), SimpleHandler)
print("Serwer uruchomiony na porcie 21000...")
httpd.serve_forever()
```

Strona HTML (`page.html`):
```html
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Licznik</title>
</head>

<body>
    <h1>Witaj w liczniku!</h1>
    <p>Stan licznika: {counter}</p>
    <p>Kliknij <a href="/">tutaj</a>, aby zwiększyć licznik.</p>
</body> 
</html>
```

### Obsługa ciasteczek w `requests`

Biblioteka **requests** zapewnia mechanizmy do obsługi ciasteczek, zarówno przy wysyłaniu żądań, jak i przy odbieraniu odpowiedzi z serwera. Wykorzystuje ona obiekt typu **CookieJar** do przechowywania ciasteczek.

#### Odbieranie ciasteczek

Po otrzymaniu odpowiedzi z serwera, `requests` automatycznie zapisuje ciasteczka w obiekcie `Response.cookies`. Dostęp do ciasteczek można uzyskać za pomocą atrybutu `cookies` obiektu `Response`. Ciasteczka są przechowywane w formie obiektu `RequestsCookieJar`, który działa jak słownik.

Jeśli serwer z przykładu powyżej jest uruchomiony, to:

In [1]:
import requests
from bs4 import BeautifulSoup

url = "http://localhost:8080/"
r = requests.get(url)
r.cookies

<RequestsCookieJar[Cookie(version=0, name='session_id', value='8670032e-474b-44e4-becd-480618abfc25', port=None, port_specified=False, domain='localhost.local', domain_specified=False, domain_initial_dot=False, path='/', path_specified=False, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={}, rfc2109=False)]>

In [2]:
r.cookies['session_id']

'8670032e-474b-44e4-becd-480618abfc25'

In [3]:
BeautifulSoup(r.content, 'html.parser').find('p').text

'Stan licznika: 1'

#### Odsyłanie ciasteczek

Ciasteczka nie są jednak automatycznie odsyłane:

In [4]:
for _ in range(5):
    r = requests.get(url)
    print(BeautifulSoup(r.content, 'html.parser').find('p').text)

Stan licznika: 1
Stan licznika: 1
Stan licznika: 1
Stan licznika: 1
Stan licznika: 1


Można je odesłać w nagłówku `Cookie` lub za pomocą parametrów `cookies` w funkcji `requests.get()`:

In [5]:
r = requests.get(url)

for _ in range(4):
    print(BeautifulSoup(r.content, 'html.parser').find('p').text)
    r = requests.get(url, cookies=r.cookies)
    
print(BeautifulSoup(r.content, 'html.parser').find('p').text)

Stan licznika: 1
Stan licznika: 2
Stan licznika: 3
Stan licznika: 4
Stan licznika: 5


In [6]:
r = requests.get(url)

for _ in range(4):
    print(BeautifulSoup(r.content, 'html.parser').find('p').text)
    r = requests.get(url, cookies={'session_id': r.cookies['session_id']})
    
print(BeautifulSoup(r.content, 'html.parser').find('p').text)

Stan licznika: 1
Stan licznika: 2
Stan licznika: 3
Stan licznika: 4
Stan licznika: 5


#### Użycie obiektu `Session`

Używając obiektu `Session` z requests, można utrzymać stan (w tym ciasteczka) przez wiele żądań. Dzięki temu, wszystkie ciasteczka są przechowywane i automatycznie dołączane do kolejnych żądań wysyłanych w ramach tej sesji.


In [7]:
session = requests.Session()

for _ in range(5):
    r = session.get(url)
    print(BeautifulSoup(r.content, 'html.parser').find('p').text)

Stan licznika: 1
Stan licznika: 2
Stan licznika: 3
Stan licznika: 4
Stan licznika: 5
